In [ ]:
!nvidia-smi

Tue Dec 20 10:33:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    51W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!python -m pip install --upgrade pip setuptools wheel
# https://colab.research.google.com/github/jakiya99/Torrent-to-Onedrive/blob/main/new_torrents_download_to_onedrive.ipynb#scrollTo=Fil_bHuAD8S3
# https://github.com/jakiyaa/Torrent-to-Onedrive/blob/main/full%20tutorial.md
# https://github.com/jakiyaa/rclone-authenticate

!wget https://downloads.rclone.org/v1.60.1/rclone-v1.60.1-linux-amd64.deb
!apt install ./rclone-v1.60.1-linux-amd64.deb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 31.5 MB/s 
     |████████████████████████████████| 1.2 MB 61.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
--2022-12-20 10:34:09--  https://downloads.rclone.org/v1.60.1/rclone-v1.60.1-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone

In [ ]:
!rclone config

2022/12/20 10:34:19 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found, make a new one?
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n

Enter name for new remote.
name> onedrive

Option Storage.
Type of storage to configure.
Choose a number from below, or type in your own value.
 1 / 1Fichier
   \ (fichier)
 2 / Akamai NetStorage
   \ (netstorage)
 3 / Alias for an existing remote
   \ (alias)
 4 / Amazon Drive
   \ (amazon cloud drive)
 5 / Amazon S3 Compliant Storage Providers including AWS, Alibaba, Ceph, China Mobile, Cloudflare, ArvanCloud, Digital Ocean, Dreamhost, Huawei OBS, IBM COS, IDrive e2, IONOS Cloud, Lyve Cloud, Minio, Netease, RackCorp, Scaleway, SeaweedFS, StackPath, Storj, Tencent COS, Qiniu and Wasabi
   \ (s3)
 6 / Backblaze B2
   \ (b2)
 7 / Better checksums for other remotes
   \ (hasher)
 8 / Box
   \ (box)
 9 / Cache a remote
   \ (cache)
10 / Citrix Sharefile
   \ (sharefile)
11 / Combine seve

In [ ]:
import os
path = "/content"
os.chdir(path)

!pwd

/content


In [ ]:
!mkdir onedrive
# 사용자 shell이 종료되어도 작동되기 위해 nohup(no hang up), & 사용
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


In [ ]:
!ls ./onedrive

# Install <img src="https://api.wandb.ai/files/cayush/images/projects/464653/2dbc1a01.png" height="35" width="170" />
https://tts.readthedocs.io/en/latest/index.html

In [ ]:
!pip install TTS==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.0/590.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.

In [ ]:
!pip install jamo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# change the current working directory
os.chdir('/content/drive/MyDrive/glow-tts-dialect')
!pwd

/content/drive/MyDrive/glow-tts-dialect


In [ ]:
# https://github.com/coqui-ai/TTS/blob/dev/TTS/tts/datasets/formatters.py
from korean.ko_cleaner import tokenize
import os

# formatter for jss (jejueo single speaker) speech dataset
def jss_formatter(root_path, meta_file, ignored_speakers=None):
  """Normalize JSS meta data file for TTS"""
  txt_file = os.path.join(root_path, meta_file)
  items = []
  with open(txt_file, "r", encoding="utf-8") as ttf:
    for line in ttf:
      cols = line.split("|")
      wav_file = os.path.join(root_path, cols[0])
      text = cols[1].strip()
      text = tokenize(text)
      items.append({"text": text, "audio_file": wav_file, "speaker_name": "", "root_path": root_path})
  return items

In [ ]:
from korean.ko_cleaner import PUNC, VALID_CHARS

print( "".join(VALID_CHARS))
print(PUNC)

ᄀᄁᄂᄃᄄᄅᄆᄇᄈᄉᄊᄋᄌᄍᄎᄏᄐᄑ하ᅢᅣᅤᅥᅦᅧᅨᅩᅪᅫᅬᅭᅮᅯᅰᅱᅲᅳᅴᅵᆞᆢᆨᆩᆪᆫᆬᆭᆮᆯᆰᆱᆲᆳᆴᆵᆶᆷᆸᆹᆺᆻᆼᆽᆾᆿᇀᇁᇂ
!'(),-.:;? 


In [ ]:
from korean.ko_cleaner import PUNC, VALID_CHARS

# 🐸Coqui Glow-TTS 
from trainer import Trainer, TrainerArgs
# from TTS.encoder.utils.training import init_training

from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig, BaseAudioConfig, CharactersConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

output_path = '/content/drive/MyDrive/glow-tts-dialect/model/glow-tts/'

# define dataset config
dataset_config = BaseDatasetConfig(dataset_name = "jss", path="/content/onedrive/jss-dataset", meta_file_train="metadata.txt")

# define audio config
audio_config = BaseAudioConfig(
    sample_rate=44100,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    preemphasis=0.0,
    ref_level_db=20,
    log_func="np.log",
    do_trim_silence=True,
    mel_fmin=0,
    mel_fmax=None,
    spec_gain=1.0,
    signal_norm=False,
    do_amp_to_db_linear=False,
)

# define charactes config
# characters_class="TTS.tts.models.vits.VitsCharacters",
characters_config = CharactersConfig(
    pad="_",
    eos="~",
    characters= "".join(VALID_CHARS),
    punctuations= PUNC,
    phonemes=None,
)

# init the training config
config = GlowTTSConfig(
    batch_size=64,
    eval_batch_size=16,
    eval_split_size=0.1,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner=None,
    use_phonemes=False,
    phoneme_language=None,
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    audio=audio_config,
    characters=characters_config,
)


# init the audio processor
ap = AudioProcessor.init_from_config(config)

# init the tokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, eval_split_size=config.eval_split_size, formatter=jss_formatter)

# init the model
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)


# init the trainer and 🚀
trainer = Trainer(TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples)

trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 10000 files in /content/onedrive/jss-dataset


 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False

 > Model has 28600849 parameters

 > EPOCH: 0/1000
 --> /content/drive/MyDrive/glow-tts-dialect/model/glow-tts/run-December-20-2022_10+36AM-309e698




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 9000



 > TRAINING (2022-12-20 10:37:41) 


 | > Preprocessing samples
 | > Max text length: 204
 | > Min text length: 24
 | > Avg text length: 65.22155555555555
 | 
 | > Max audio length: 813590.0
 | > Min audio length: 48662.0
 | > Avg audio length: 219195.92444444445
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


...

 | > Preprocessing samples
 | > Max text length: 204
 | > Min text length: 24
 | > Avg text length: 65.22155555555555
 | 
 | > Max audio length: 813590.0
 | > Min audio length: 48662.0
 | > Avg audio length: 219195.92444444445
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 9/141 -- GLOBAL_STEP: 150
     | > loss: 4.97950  (4.85998)
     | > log_mle: 0.86638  (0.85662)
     | > loss_dur: 4.11312  (4.00336)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 12.15284  (12.08806)
     | > current_lr: 0.00000 
     | > step_time: 0.32440  (0.35875)
     | > loader_time: 0.00420  (4.06031)


   --> STEP: 34/141 -- GLOBAL_STEP: 175
     | > loss: 5.30685  (5.08643)
     | > log_mle: 0.88287  (0.87042)
     | > loss_dur: 4.42398  (4.21601)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 12.56804  (12.35571)
     | > current_lr: 0.00000 
     | > step_time: 0.35500  (0.44526)
     | > loader_time: 0.00530  (41.23487)


   --> STEP: 59/141 -- GLOBAL_STEP: 200
     | > loss: 5.42902  (5.17605)
     | > log_mle: 0.88638  (0.87701)
     | > loss_dur: 4.54264  (4.29904)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 12.72318  (12.45279)
     | > current_lr: 0.00000 
     | > step_time: 0.38770  (0

['ᄋ', 'ᅡ', ',', ' ', 'ᄌ', 'ᅦ', 'ᆺ', 'ᄀ', 'ᅮ', 'ᆨ', 'ᄋ', 'ᅳ', 'ᄅ', 'ᅩ', ' ', 'ᄒ', 'ᅦ', 'ᄉ', 'ᅥ', '.', ' ', 'ᄋ', 'ᅰ', 'ᄂ', 'ᅣ', 'ᄒ', 'ᅥ', 'ᄆ', 'ᅧ', 'ᆫ', ' ', 'ᄋ', 'ᅮ', 'ᄅ', 'ᅧ', 'ᄂ', 'ᅢ', 'ᄌ', 'ᅦ', 'ᆫ', ' ', 'ᄒ', 'ᅥ', 'ᄆ', 'ᅵ', 'ᆫ', ' ', 'ᄆ', 'ᅦ', 'ᆺ', ' ', 'ᄇ', 'ᅮ', 'ᆯ', 'ᄋ', 'ᅳ', 'ᆯ', ' ', 'ᄋ', 'ᅮ', 'ᄅ', 'ᅧ', 'ᄂ', 'ᅢ', 'ᄌ', 'ᅵ', ' ', 'ᄋ', 'ᅡ', 'ᄂ', 'ᅧ', 'ᆷ', 'ᄂ', 'ᅵ', ' ', 'ᄁ', 'ᅡ', 'ᅌ', 'ᅨ', '?', ' ', 'ᄋ', 'ᅨ', 'ᆺ', 'ᄂ', 'ᅡ', 'ᆯ', ' ', 'ᄋ', 'ᅥ', 'ᄅ', 'ᅳ', 'ᆫ', 'ᄃ', 'ᅳ', 'ᆯ', 'ᄋ', 'ᅳ', 'ᆫ', ' ', 'ᄌ', 'ᅦ', 'ᆺ', 'ᄀ', 'ᅮ', 'ᆨ', 'ᄋ', 'ᅳ', 'ᄅ', 'ᅩ', ' ', 'ᄒ', 'ᅦ', 'ᆺ', 'ᄀ', 'ᅮ', 'ᄂ', 'ᅡ', '?', ' ', 'ᄋ', 'ᅡ', 'ᄋ', 'ᅡ', '.', '~']['ᄋ', 'ᅡ', ',', ' ', 'ᄌ', 'ᅦ', 'ᆺ', 'ᄀ', 'ᅮ', 'ᆨ', 'ᄋ', 'ᅳ', 'ᄅ', 'ᅩ', ' ', 'ᄒ', 'ᅦ', 'ᄉ', 'ᅥ', '.', ' ', 'ᄋ', 'ᅰ', 'ᄂ', 'ᅣ', 'ᄒ', 'ᅥ', 'ᄆ', 'ᅧ', 'ᆫ', ' ', 'ᄋ', 'ᅮ', 'ᄅ', 'ᅧ', 'ᄂ', 'ᅢ', 'ᄌ', 'ᅦ', 'ᆫ', ' ', 'ᄒ', 'ᅥ', 'ᄆ', 'ᅵ', 'ᆫ', ' ', 'ᄆ', 'ᅦ', 'ᆺ', ' ', 'ᄇ', 'ᅮ', 'ᆯ', 'ᄋ', 'ᅳ', 'ᆯ', ' ', 'ᄋ', 'ᅮ', 'ᄅ', 'ᅧ', 'ᄂ', 'ᅢ', 'ᄌ', 'ᅵ', ' ', 'ᄋ', 'ᅡ', 'ᄂ', 'ᅧ', 'ᆷ', 'ᄂ', 'ᅵ',



   --> STEP: 134/141 -- GLOBAL_STEP: 275
     | > loss: 5.33584  (5.26972)
     | > log_mle: 0.90932  (0.88839)
     | > loss_dur: 4.42652  (4.38132)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 12.59052  (12.54626)
     | > current_lr: 0.00000 
     | > step_time: 1.51140  (0.56777)
     | > loader_time: 0.01680  (45.74975)

   --> STEP: 134/141 -- GLOBAL_STEP: 275
     | > loss: 5.33584  (5.26972)
     | > log_mle: 0.90932  (0.88839)
     | > loss_dur: 4.42652  (4.38132)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 12.59052  (12.54626)
     | > current_lr: 0.00000 
     | > step_time: 1.51140  (0.56777)
     | > loader_time: 0.01680  (45.74975)


 > EVALUATION 


 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 34 not found characters:
	| > I
	| > t
	| > o
	| > k
	| > m
	| > e
	| > q
	| > u
	| > i
	| > a
	| > l
	| > n
	| > g
	| > d
	| > v
	| > p
	| > c
	| > w
	| > h
	| > b
	| > s
	| > B
	| > r
	| > y
	| > D
	| > f
	| > T
	| > P
	| > N
	| > 2
	| > 1
	| > 9
	| > 6
	| > 3
| > Number of instances : 1000
 | > Preprocessing samples
 | > Max text length: 195
 | > Min text length: 29
 | > Avg text length: 65.404
 | 
 | > Max audio length: 756246.0
 | > Min audio length: 65558.0
 | > Avg audio length: 218129.435
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 34 not found characters:
	| > I
	| > t
	| > o
	| > k
	| > m
	| > e
	| > q
	| > u
	| > i
	| > a
	| > l
	| > n
	| > g
	| > d
	| > v
	| > p
	| > c
	| > w
	| > h
	| > b
	| > s
	| > B
	| > r
	| 